TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [279]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [280]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [281]:
#tag_name = 'all_transcripts'
tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 50

In [282]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [283]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [284]:
# all_chars = sorted(list(set(corpus)))

# #remove most punctuation, but leave spaces, exclamation points, and periods
# list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

# chars = [character for character in all_chars if character not in list_to_remove]
# print(chars)
# # hat tip to this post for the idea of removing one list from another
# # https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

In [285]:
# # remove all unwanted characters from the original corpus
# corpus_list = [character for character in corpus if character not in list_to_remove]

# # Re-join all letters back into a single string as the updated corpus
# corpus = ''.join(corpus_list)

In [286]:
# If running models with punctuation removed, leave commented out
chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 74


In [287]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '"': 2, '%': 3, "'": 4, ',': 5, '-': 6, '.': 7, '/': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ';': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, '[': 45, ']': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '—': 73}


In [288]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '"', 3: '%', 4: "'", 5: ',', 6: '-', 7: '.', 8: '/', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: ';', 20: '?', 21: 'A', 22: 'B', 23: 'C', 24: 'D', 25: 'E', 26: 'F', 27: 'G', 28: 'H', 29: 'I', 30: 'J', 31: 'K', 32: 'L', 33: 'M', 34: 'N', 35: 'O', 36: 'P', 37: 'Q', 38: 'R', 39: 'S', 40: 'T', 41: 'U', 42: 'V', 43: 'W', 44: 'Y', 45: '[', 46: ']', 47: 'a', 48: 'b', 49: 'c', 50: 'd', 51: 'e', 52: 'f', 53: 'g', 54: 'h', 55: 'i', 56: 'j', 57: 'k', 58: 'l', 59: 'm', 60: 'n', 61: 'o', 62: 'p', 63: 'q', 64: 'r', 65: 's', 66: 't', 67: 'u', 68: 'v', 69: 'w', 70: 'x', 71: 'y', 72: 'z', 73: '—'}


In [289]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 62,064 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [290]:
corpus[0:50]

'So I want to start by offering you a free no-tech '

In [291]:
corpus[1:41]

'o I want to start by offering you a free'

In [292]:
corpus[41]

' '

In [293]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [294]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[39, 61, 0, 29, 0, 69, 47, 60, 66, 0, 66, 61, 0, 65, 66, 47, 64, 66, 0, 48, 71, 0, 61, 52, 52, 51, 64, 55, 60, 53, 0, 71, 61, 67, 0, 47, 0, 52, 64, 51, 51, 0, 60, 61, 6, 66, 51, 49, 54, 0]
[61, 0, 29, 0, 69, 47, 60, 66, 0, 66, 61, 0, 65, 66, 47, 64, 66, 0, 48, 71, 0, 61, 52, 52, 51, 64, 55, 60, 53, 0, 71, 61, 67, 0, 47, 0, 52, 64, 51, 51, 0, 60, 61, 6, 66, 51, 49, 54, 0, 58]


In [295]:
print ( y_numeric_list[0:2])

[58, 55]


In [296]:
indices_char[51]

'e'

In [297]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [298]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 62,014 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [299]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [300]:
X.shape

(62014, 50, 1)

In [301]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [302]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [303]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [304]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [305]:

checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)
latest

In [306]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


# #model = load_model(f'{latest}')

model = load_model(f'{root_path}models/ted_model_{tag_name}')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 1600)          10252800  
_________________________________________________________________
dropout (Dropout)            (None, 50, 1600)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               4327424   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 74)                37962     
Total params: 14,618,186
Trainable params: 14,618,186
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [307]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 1182 sequence.
[47, 57, 51, 5, 0, 61, 64, 0, 66, 54, 51, 0, 58, 47, 49, 57, 0, 61, 52, 0, 47, 0, 54, 47, 60, 50, 65, 54, 47, 57, 51, 5, 0, 49, 47, 60, 0, 54, 47, 68, 51, 0, 67, 65, 0, 66, 47, 58, 57, 55]


Convert this back to text so we can see what it looks like.

In [308]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'ake, or the lack of a handshake, can have us talki'

In [309]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

ake, or the lack of a handshake, can have us talki
ke, or the lack of a handshake, can have us talkin
e, or the lack of a handshake, can have us talking
, or the lack of a handshake, can have us talking 
 or the lack of a handshake, can have us talking f
or the lack of a handshake, can have us talking fo
r the lack of a handshake, can have us talking for
 the lack of a handshake, can have us talking for 
the lack of a handshake, can have us talking for w
he lack of a handshake, can have us talking for we
e lack of a handshake, can have us talking for wee
 lack of a handshake, can have us talking for week
lack of a handshake, can have us talking for weeks
ack of a handshake, can have us talking for weeks 
ck of a handshake, can have us talking for weeks a


To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [310]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 74)
Prediction:
[[2.9541208e-10 9.6525567e-16 7.9424404e-21 1.0855082e-19 2.7164815e-20
  1.8438389e-17 1.2067170e-13 6.9280077e-25 1.4997309e-21 9.3608292e-23
  2.4874060e-21 1.7471060e-22 9.4940041e-29 4.3486421e-23 2.3189979e-26
  5.2473221e-24 9.3908270e-25 9.0857291e-19 1.9906421e-23 1.3821897e-16
  9.4517438e-25 1.5249369e-19 7.0459632e-20 1.1777280e-27 5.6942169e-25
  1.0141605e-22 7.9025056e-24 2.2374157e-21 5.7551690e-24 1.0716147e-18
  6.4498372e-24 1.6031882e-19 2.7740628e-23 4.2114049e-25 3.7186098e-24
  2.1004559e-20 6.4381705e-21 4.8346437e-19 2.3215215e-25 4.3186209e-23
  2.1447982e-24 1.6673300e-21 7.7634949e-19 5.2092168e-21 1.0501696e-27
  2.6120150e-21 3.5050761e-21 5.8899779e-08 2.0196485e-10 2.0735241e-12
  1.9711500e-12 6.2940188e-11 2.0103983e-13 9.2627620e-14 9.2792228e-18
  4.9879202e-12 6.1512947e-21 1.1625548e-14 2.2081188e-15 2.5763190e-11
  1.0000000e+00 5.6508253e-12 5.1257165e-10 4.8590497e-19 1.4510873e-10
  4.5890175e-12 2.42372

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [311]:
np.argmax(prediction)

60

So the first letter predicted after the seed sequence is:

In [312]:
indices_char[np.argmax(prediction)]

'n'

In [313]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(500):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [314]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['ake, or the lack of a handshake, can have us talki->ng for weeks and weeks and weeks. Even the BBC and The New York',
 'Times. So obviously when we think about nonverbal behavior, or body language — but we call it nonverbals as social',
 "scientists — it's language, so we think about communication. When we think about communication, we think about",
 "interactions. So what is your body language communicating to me? What's mine communicating to you? And there's a lot of",
 'reason to believe that this is a valid way to look at this. So social scientists have spen']

In [315]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['ake, or the lack of a handshake, can have us talking for weeks and weeks and weeks. Even the BBC and The New York Times.',
 'So obviously when we think about nonverbal behavior, or body language — but we call it nonverbals as social scientists —',
 "it's language, so we think about communication. When we think about communication, we think about interactions. So what",
 "is your body language communicating to me? What's mine communicating to you? And there's a lot of reason to believe that",
 'this is a valid way to look at this. So social scientists have spent a lot of time looking at the effects of our body',
 "language, or other people's body language, on judgments. And we make sweeping judgments and inferences from body",
 'language. And those judgments can predict really meaningful life outcomes like who we hire or promote, who we ask out on',
 'a date. For example, Nalini Ambady, a researcher at Tufts University, shows that when people watch 30-second soundless',
 "clips of rea